# Revised case normalization for Hirslanden Salem 2018

Notes: 
- There is no bfs dataset from Hirslanden Salem 2018 in the database. Therefore, no cases can get matched. 
- There are no CHOP codes contained in the revised cases dataset. 





In [1]:
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')

import os

from py.global_configs import *
from py.normalize import normalize

from src.service import bfs_cases_db_service as bfs_db
from src.utils.dataframe_utils import remove_duplicated_chops

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:
file_info = FileInfo(
    os.path.join(ROOT_DIR, 'raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx'),
    'Hirslanden Salem',
    '2018',
    ['Änderungen _SA_2018']
)

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db='Hirslanden Salem', year='2018', sheets=['Änderungen _SA_2018'])


In [3]:
cols_to_rename = dict(COLUMNS_TO_RENAME)
# Replace 'admno' with 'fall nummer'
cols_to_rename.pop('admno')
cols_to_rename['fall nummer'] = CASE_ID_COL

df_revised_case_d2d = normalize(file_info, 0)

2022-10-14 11:46:19.224 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 0: discarded ICDs after validation {''}
2022-10-14 11:46:19.225 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 1: discarded ICDs after validation {''}
2022-10-14 11:46:19.225 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 2: discarded ICDs after validation {''}
2022-10-14 11:46:19.226 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 5: discarded ICDs after validation {''}
2022-10-14 11:46:19.227 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 6: discarded ICDs after validation {''}
2022-10-14 11:46:19.227 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 7: discarded ICDs after validation {''}
2022-10-14 11:46:19.228 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 13: discarded ICDs after validation {''}
2022-10-14 11:46:19.229 | DEBUG    | src.utils.dataframe_utils:_validate_ic

Read 46 cases for Hirslanden Salem 2018
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
old_pd              string
new_pd              string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


2022-10-14 11:46:19.238 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 34: discarded ICDs after validation {''}
2022-10-14 11:46:19.238 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 37: discarded ICDs after validation {''}
2022-10-14 11:46:19.240 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 38: discarded ICDs after validation {''}
2022-10-14 11:46:19.240 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 39: discarded ICDs after validation {''}
2022-10-14 11:46:19.241 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 40: discarded ICDs after validation {''}
2022-10-14 11:46:19.241 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 41: discarded ICDs after validation {''}
2022-10-14 11:46:19.242 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 42: discarded ICDs after validation {''}
2022-10-14 11:46:19.243 | DEBUG    | src.utils.dataframe_utils:_valid

In [4]:
df_revised_case_d2d.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,0041602986,90C3E16484AD653A,M,88,12,3,I43B,M171,M171,M200,[N183],[],[],[]
1,0041637726,D774DF60034ACAA4,W,53,19,3,I09C,T842,T842,M200,"[K564, T811]",[],[],[]
2,0041641561,4A03EA9845D13800,M,81,9,3,L60C,N1793,N1793,M100,[],[],[],[]
3,0041643625,E3CD9E9C233B926F,M,83,10,3,F13C,E1172,E1172,M100,[M8617],[M8697],[],[]
4,0041644263,80BD3429C23B3A1E,W,76,10,3,T60E,A418,A418,M200,"[J9600, B962]",[J9699],[],[]


# Match revised DtoD data with the bfs data from the database


In [5]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['0041602986', '0041637726', '0041641561', '0041643625',
       '0041644263', '0041649868', '0041656883', '0041683635',
       '0041689417', '0041695908', '0041702122', '0041710615',
       '0041711681', '0041715963', '0041735592', '0041760287',
       '0041763118', '0041776339', '0041779228', '0041779882',
       '0041788723', '0041800898', '0041812167', '0041813832',
       '0041821441', '0041869626', '0041870623', '0041638807',
       '0041916526', '0041918398', '0041630555', '0041881668',
       '0041898735', '0041675045', '0041691601', '0041742275',
       '0041785404', '0041789235', '0041679610', '0041823802',
       '0041826915', '0041830184', '0041837846', '0041850795',
       '0041869375', '0041927781'], dtype=object)

In [6]:
# change the case_id the same format as bfs data e.g. '0041194601'
# revised_case_id_pad0 = [case_id.zfill(10) for case_id in revised_case_id]
# revised_case_id_pad0[0]
# df_revised_case_d2d['case_id'] = revised_case_id_pad0

In [7]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids(revised_case_id)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl


In [8]:
# 
print('There are {} out of {} revised cases from DtoD that are matched with the database for {} {}'.format(len(revised_case_db), len(revised_case_id), file_info.hospital_name_db, file_info.year))

There are 0 out of 46 revised cases from DtoD that are matched with the database for Hirslanden Salem 2018


In [9]:
# if we find the match cases, then we need to check if the case_id, gender, year....are match

In [10]:
df_revised_case_d2d.columns

Index(['case_id', 'patient_id', 'gender', 'age_years', 'duration_of_stay',
       'pccl', 'drg', 'old_pd', 'new_pd', 'bfs_code', 'added_icds',
       'removed_icds', 'added_chops', 'removed_chops'],
      dtype='object')

In [11]:
# substract the columns [case_id, new_pd_bfs_code	added_icds	removed_icds	added_chops	removed_chops] from revised_cases 

# revised_case_d2d_subset = df_revised_case_d2d[['case_id', 'new_pd', 'added_icds', 'removed_icds', 'added_chops', 'removed_chops']]
# revised_case_d2d_subset.head()

,case_id,new_pd,added_icds,removed_icds,added_chops,removed_chops
0,0041602986,M171,[N183],[],[],[]
1,0041637726,T842,"[K564, T811]",[],[],[]
2,0041641561,N1793,[],[],[],[]
3,0041643625,E1172,[M8617],[M8697],[],[]
4,0041644263,A418,"[J9600, B962]",[J9699],[],[]


In [12]:
# merge revised_case_d2d_subset with revised_case_db that matched from bfs data
# revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='right', on='case_id')
# revised_case_normalized.head()

,case_id,new_pd,added_icds,removed_icds,added_chops,removed_chops,drg_cost_weight,aimedic_id,hospital_id,patient_id,...,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl
0,0041602986,M171,[N183],[],[],[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0041637726,T842,"[K564, T811]",[],[],[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0041641561,N1793,[],[],[],[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0041643625,E1172,[M8617],[M8697],[],[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0041644263,A418,"[J9600, B962]",[J9699],[],[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0041649868,S7208,"[I360, I5001, R31]",[],[],[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0041656883,J100,[J9600],[],[],[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0041683635,S723,"[L8914, D6833]",[],[],[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0041689417,N328,"[F009, G309]",[F03],[],[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0041695908,J209,[I5013],[I5019],[],[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
